## E-commerce EDA

In [0]:
# load website_sessions
website_sessions_df = spark.table("workspace.default.website_sessions")
display(website_sessions_df.head(2))

website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
1,2012-03-19T08:04:16.000Z,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
2,2012-03-19T08:16:49.000Z,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


In [0]:
print(f"Total rows: {website_sessions_df.count():,}")
print(f"Total columns: {len(website_sessions_df.columns)}")
print("\nColumns:", website_sessions_df.columns)

Total rows: 472,871
Total columns: 9

Columns: ['website_session_id', 'created_at', 'user_id', 'is_repeat_session', 'utm_source', 'utm_campaign', 'utm_content', 'device_type', 'http_referer']


In [0]:
# summary stats
display(website_sessions_df.describe())

summary,website_session_id,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
count,472871,472871,472871,389543,389543,389543,472871,432954
mean,236436.0,198037.97016311003,0.166119301035589,null,null,null,null,null
stddev,136506.24390847475,111992.99779840393,0.37218808659258484,null,null,null,null,null
min,1,1,0,bsearch,brand,b_ad_1,desktop,https://www.bsearch.com
max,472871,394318,1,socialbook,pilot,social_ad_2,mobile,https://www.socialbook.com


### Find top traffic source

In [0]:
%sql
-- when the website first launched, look into what traffic is bringing more website sessions
SELECT utm_source, utm_campaign, http_referer,  COUNT(DISTINCT website_session_id) AS sessions
FROM website_sessions
WHERE created_at BETWEEN '2012-03-11' AND '2012-04-11'
GROUP BY 1, 2, 3
ORDER BY 4 DESC

utm_source,utm_campaign,http_referer,sessions
gsearch,nonbrand,https://www.gsearch.com,3421
null,null,null,24
null,null,https://www.gsearch.com,24
gsearch,brand,https://www.gsearch.com,23
bsearch,brand,https://www.bsearch.com,7
null,null,https://www.bsearch.com,7


#### Traffic source conversion rate

In [0]:
%sql
-- calculate the conversion rate (CVR) from session to order; focus on gsearch, nonbrand
SELECT 
  COUNT(DISTINCT w.website_session_id) AS sessions, 
  COUNT(DISTINCT order_id) AS orders,
  ROUND(COUNT(DISTINCT order_id) / COUNT(DISTINCT w.website_session_id), 2) AS session_to_order_conv_rate
FROM website_sessions w LEFT JOIN orders o ON w.website_session_id = o.website_session_id
WHERE w.created_at < '2012-04-14' AND utm_source = 'gsearch' AND utm_campaign = 'nonbrand';

sessions,orders,session_to_order_conv_rate
3895,112,0.03


#### Traffic source trending

In [0]:
%sql
-- pull gsearch nonbrand trended session volume by week
-- the company bid down gsearch nonbrand on 2012-04-15

SELECT 
  DATE(DATE_TRUNC('WEEK', created_at)) AS weekly_session, 
  COUNT(DISTINCT website_session_id) AS sessions
FROM website_sessions
WHERE created_at BETWEEN '2012-03-10' AND '2012-05-10'
AND utm_source='gsearch' AND utm_campaign = 'nonbrand'
GROUP BY DATE_TRUNC('WEEK', created_at)
-- the nonbrand seems to be sensitive to the bid changes and volume is going down

weekly_session,sessions
2012-03-19,965
2012-03-26,980
2012-04-02,1138
2012-04-09,967
2012-04-16,611
2012-04-23,589
2012-04-30,682
2012-05-07,350


#### Bid optimization for paid traffic

In [0]:
%sql
-- pull conversion rates from session to order, by device type
-- focus only on gsearch and nonbrand
SELECT 
  device_type, 
  COUNT(DISTINCT w.website_session_id) AS sessions, 
  COUNT(DISTINCT order_id) AS orders,
  ROUND(COUNT(DISTINCT order_id)  / COUNT(DISTINCT w.website_session_id), 2) AS cvr
FROM website_sessions w LEFT JOIN orders o ON w.website_session_id = o.website_session_id
WHERE utm_source = 'gsearch' AND utm_campaign = 'nonbrand'
AND w.created_at < '2012-05-11'
GROUP BY 1;
-- should increase bids on desktop

device_type,sessions,orders,cvr
mobile,2492,24,0.01
desktop,3911,146,0.04


### Product analysis

In [0]:
%sql
-- will launch a new product but we want to do a deep dive on our current flagship product
-- pull monthly trends to date for number of sales, total revenue, and total margin generated for the business
-- < 2013-01-04

SELECT 
    YEAR(created_at) AS yr, 
    MONTH(created_at) AS mo, 
    COUNT(DISTINCT order_id) AS number_of_sales, 
    ROUND(SUM(price_usd),2) AS total_revenue, 
    ROUND(SUM(price_usd - cogs_usd),2) AS total_margin
FROM orders
WHERE created_at < '2013-01-04'
GROUP BY 1, 2
ORDER BY 1, 2

yr,mo,number_of_sales,total_revenue,total_margin
2012,3,60,2999.4,1830.0
2012,4,99,4949.01,3019.5
2012,5,108,5398.92,3294.0
2012,6,140,6998.6,4270.0
2012,7,169,8448.31,5154.5
2012,8,228,11397.72,6954.0
2012,9,287,14347.13,8753.5
2012,10,371,18546.29,11315.5
2012,11,618,30893.82,18849.0
2012,12,506,25294.94,15433.0


#### Analyze new product launch

In [0]:
%sql
-- we launched our second product back on Jan 6th, 2013.
-- we want to see monthly order volume, overall conversion rates, revenue per session, and a breakdown of sales by product
-- 2012-04-01 - 2013-04-05

SELECT 
  YEAR(o.created_at) AS yr, 
  MONTH(o.created_at) AS mo, 
  COUNT(DISTINCT order_id) AS orders, 
  ROUND(COUNT(DISTINCT order_id) / COUNT(DISTINCT w.website_session_id), 2) AS cvr,
  ROUND(SUM(price_usd) / COUNT(DISTINCT w.website_session_id), 2) AS revenue_per_session,
  COUNT(DISTINCT CASE WHEN primary_product_id=1 THEN order_id END) AS product_one_orders, 
  COUNT(DISTINCT CASE WHEN primary_product_id=2 THEN order_id END) AS product_two_orders
FROM website_sessions w LEFT JOIN orders o ON w.website_session_id = o.website_session_id
WHERE o.created_at BETWEEN '2012-04-01' AND '2013-04-05'
GROUP BY 1, 2
ORDER BY 1, 2;

-- cvr and revenue per session have been improving
-- but we cannot tell if the growth since Jan is due to our new product launch or just a continuation of our overall business improvments

yr,mo,orders,cvr,revenue_per_session,product_one_orders,product_two_orders
2012,4,99,1.0,49.99,99,0
2012,5,108,1.0,49.99,108,0
2012,6,140,1.0,49.99,140,0
2012,7,169,1.0,49.99,169,0
2012,8,228,1.0,49.99,228,0
2012,9,287,1.0,49.99,287,0
2012,10,371,1.0,49.99,371,0
2012,11,618,1.0,49.99,618,0
2012,12,506,1.0,49.99,506,0
2013,1,390,1.0,51.2,343,47


### Device analysis

In [0]:
from pyspark.sql.functions import count, countDistinct

result = website_sessions_df.groupBy('device_type').agg(
    count("website_session_id").alias("total_sessions"),
    countDistinct("user_id").alias("total_users")
)

display(result)

device_type,total_sessions,total_users
mobile,145844,133839
desktop,327027,288580


#### Trend pattern by device type

In [0]:
%sql
-- we bid our gsearch nonbrand desktop campaigns up on 2012-05-19
-- now pull weekly trends for both desktop and mobile so we can see the impact on volume
SELECT 
  DATE(DATE_TRUNC('WEEK', w.created_at)) AS weekly_session, 
  COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN w.website_session_id END) AS desktop_sessions,
  COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN o.order_id END) AS desktop_orders,
  ROUND(COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN o.order_id END) / COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN w.website_session_id END), 3)  AS desktop_cvr,
  COUNT(DISTINCT CASE WHEN device_type = 'mobile' THEN w.website_session_id END) AS mobile_sessions,
  COUNT(DISTINCT CASE WHEN device_type = 'mobile' THEN o.order_id END) AS mobile_orders,
  ROUND(COUNT(DISTINCT CASE WHEN device_type = 'mobile' THEN o.order_id END) / COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN w.website_session_id END), 3)  AS mobile_cvr
FROM website_sessions w LEFT JOIN orders o ON w.website_session_id = o.website_session_id
WHERE 
  w.created_at BETWEEN '2012-04-15' AND '2012-06-09'
  AND utm_source = 'gsearch' AND utm_campaign = 'nonbrand'
GROUP BY 1
ORDER BY 1;

-- -- mobile sessions and orders dropped while desktop sessions and orders were strong after the bid optimization --> right direction!


weekly_session,desktop_sessions,desktop_orders,desktop_cvr,mobile_sessions,mobile_orders,mobile_cvr
2012-04-09,38,2,0.053,25,2,0.053
2012-04-16,382,12,0.031,229,4,0.01
2012-04-23,353,11,0.031,236,1,0.003
2012-04-30,424,11,0.026,258,2,0.005
2012-05-07,431,17,0.039,281,1,0.002
2012-05-14,437,17,0.039,213,3,0.007
2012-05-21,644,22,0.034,188,0,0.0
2012-05-28,579,26,0.045,178,3,0.005
2012-06-04,541,24,0.044,146,1,0.002


### Identify repeat users

In [0]:
%sql
-- identify repeat users: pull data on how many of our website visitors come back for another session; 2014-2014/11/01
-- count users by the number of repeart sessions they have
-- identify when was their first session
WITH user_table AS (
	SELECT user_id, MIN(created_at) AS first_session
	FROM website_sessions
	WHERE created_at BETWEEN '2014-01-01' AND '2014-11-01'
	GROUP BY 1
),
-- create repeat session count
repeat_sessions AS (
	SELECT u.user_id, 
	COUNT(DISTINCT CASE WHEN w.created_at > first_session THEN website_session_id ELSE NULL END) AS repeat_sessions
	FROM website_sessions w RIGHT JOIN user_table u 
		ON w.user_id = u.user_id AND created_at BETWEEN '2014-01-01' AND '2014-11-01' -- apply filter during ON to keep users with 0 repeat sessions
	GROUP BY 1
)
SELECT DISTINCT repeat_sessions, COUNT(DISTINCT user_id) as USERS
FROM repeat_sessions
GROUP BY 1
ORDER BY 1;

repeat_sessions,USERS
0,128424
1,14391
2,875
3,4686


#### Check time between second and first session

In [0]:
%sql
WITH sessions AS (
	SELECT *, ROW_NUMBER()OVER(PARTITION BY user_id ORDER BY created_at) AS rn
	FROM website_sessions
    WHERE created_at BETWEEN '2014-01-01' AND '2014-11-03'
),
users AS (
	SELECT user_id, 
    MAX(CASE WHEN rn=1 THEN created_at END) AS first_session, -- RETRIEVE FIRST SESSION DATE
    MAX(CASE WHEN rn=2 THEN created_at END) AS second_session -- RETRIEVE SECOND SESSION DATE
	FROM sessions
	GROUP BY 1
    HAVING second_session IS NOT NULL -- having can refer to the newly crated alias
)
SELECT 
	ROUND(AVG(DATEDIFF(second_session, first_session)), 2) AS avg_days, 
	ROUND(MIN(DATEDIFF(second_session, first_session)), 2) AS min_days, 
	ROUND(MAX(DATEDIFF(second_session, first_session)), 2) AS max_days
FROM users

avg_days,min_days,max_days
32.61,1,69


#### Analyze new sessions vs. repeat sessions channel

In [0]:
%sql

-- understand the channels they come back through. curious if it's all direct type-in, or if we're paying for these cutomers with paid search ads multiple times
-- compare new vs. repeat sessions by channel; 2014 to 11/05/2014

-- check types of channel
-- utm_source NULL http_refer NULL: direct_type_in
-- utm_source NULL http_refer NOT NULL: organic_search
-- paid_brand: utm_source IN ('bsearch', 'gsearch'), utm_campaign = 'brand'
-- paid_nonbrand: utm_source IN ('bsearch', 'gsearch'), utm_campaign = 'nonbrand'
-- paid_social: utm_source = 'socialbook'

WITH session_table AS (
	SELECT user_id, 
		ROW_NUMBER()OVER(PARTITION BY user_id ORDER BY created_at) AS rn,
        CASE 
			WHEN utm_source IS NULL AND http_referer IS NULL THEN 'direct_typ_in'
			WHEN utm_source IS NULL AND http_referer IS NOT NULL THEN 'organic_search'
			WHEN utm_source IN ('bsearch', 'gsearch') AND utm_campaign = 'brand' THEN 'paid_brand'
			WHEN utm_source IN ('bsearch', 'gsearch') AND utm_campaign = 'nonbrand' THEN 'paid_nonbrand'
			WHEN utm_source = 'socialbook' THEN 'paid_social'
         ELSE 'other'
         END AS channel_group
	FROM website_sessions
	WHERE created_at BETWEEN '2014-01-01' AND '2014-11-05'
)
SELECT 
	channel_group, 
	COUNT(DISTINCT CASE WHEN rn=1 THEN user_id END) AS new_sessions, 
	COUNT(DISTINCT CASE WHEN rn=2 THEN user_id END) AS second_sessions,
	ROUND(COUNT(DISTINCT CASE WHEN rn=2 THEN user_id END)*100/ COUNT(DISTINCT CASE WHEN rn=1 THEN user_id END), 2) AS repeat_session_rate,
    COUNT(DISTINCT user_id) AS total_users_with_1or2_sessions,
    COUNT(DISTINCT CASE WHEN rn=1 THEN user_id END) - COUNT(DISTINCT CASE WHEN rn=2 THEN user_id END) AS user_with_1_session
FROM session_table
WHERE rn in (1, 2)
GROUP BY 1
ORDER BY repeat_session_rate DESC;

channel_group,new_sessions,second_sessions,repeat_session_rate,total_users_with_1or2_sessions,user_with_1_session
paid_brand,7265,6726,92.58,13641,539
direct_typ_in,7392,6528,88.31,13578,864
organic_search,7981,6992,87.61,14541,989
paid_nonbrand,119950,0,0.0,119950,119950
paid_social,7652,0,0.0,7652,7652


#### CVR, revenue per session

In [0]:
%sql
-- a comparison of conversion rates and revenue per session for repeat sessions vs. new sessions
-- use 2014-01-01 to 2014-11-08
SELECT 
  is_repeat_session, 
  COUNT(DISTINCT w.website_session_id) AS sessions, 
  ROUND(COUNT(DISTINCT order_id) / COUNT(DISTINCT w.website_session_id), 3) AS cvr,
  ROUND(SUM(price_usd) / COUNT(DISTINCT w.website_session_id), 3) AS rev_per_session
FROM website_sessions w LEFT JOIN orders o ON w.website_session_id = o.website_session_id
WHERE w.created_at BETWEEN '2014-01-01' AND '2014-11-08'
GROUP BY 1

is_repeat_session,sessions,cvr,rev_per_session
0,149787,0.068,4.344
1,33577,0.081,5.169
